In [1]:
from openai import OpenAI
from docx import Document

In [2]:

# filepath: /path/to/your/file.py
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

In [3]:
def convert_doc_to_list():
    # Open the Word document
    document = Document("soguk_savas_soruları.docx")
    qa_list = []
    #qa_dict = {}
    
    # Read through the document paragraph by paragraph
    for i in range(len(document.paragraphs) - 1):
        # Extract the question
        if document.paragraphs[i].text.startswith("Q:"):
            question = document.paragraphs[i].text[2:].strip()
            
            # Extract the answer
            if document.paragraphs[i + 1].text.startswith("A:"):
                answer = document.paragraphs[i + 1].text[2:].strip()
                # Add to the dictionary
                qa_list.append(f"{question}: {answer}")
                
    return qa_list

# Example usage
qa_list = convert_doc_to_list()

# save as pickle    
import pickle
with open('qa_list.pkl', 'wb') as f:
    pickle.dump(qa_list, f)
# Print the list
print(qa_list[:2])

['Dilek tutmana yardımcı olacak meyve nedir?: WishNe', 'Asla gelmediği için izleyemediğimiz Coen Biraderler filmi nedir?: Yurtiçi Kargo']


In [4]:
system_prompt = """
Convert following q-a pairs about dad jokes to a chatbot conversation:
1. System prompt is "You are a dad joke expert. Create dad jokes in accordance with the user input."
2. Create a user input for the q-a pairs. 
3. Put q-a pairs in proper format. 

Here is a few examples:
q-a pair:
'Dilek tutmana yardımcı olacak meyve nedir?': 'WishNe'
structured-chatbot-conversation:
{
    "messages": [
        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},
        {"role": "user", "content": "Bana bir espri yapar mısın?"},
        {"role": "assistant", "content": "Dilek tutmana yardımcı olacak meyve nedir?"},
        {"role": "user", "content": "Nedir?"},
        {"role": "assistant", "content": "WishNe."}
    ]
}
q-a pair:
'Asla gelmediği için izleyemediğimiz Coen Biraderler filmi nedir?': 'Yurtiçi Kargo'
structured-chatbot-conversation:
{
    "messages": [
        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},
        {"role": "user", "content": "Bana yurtiçi kargonun yavaşlığı  ile ilgili bir espri yapar mısın?"},
        {"role": "assistant", "content": "Asla gelmediği için izleyemediğimiz Coen Biraderler filmi nedir?"},
        {"role": "user", "content": "Nedir?"},
        {"role": "assistant", "content": "Yurtiçi Kargo."}
    ]
}
q-a pair:
'Adı Chaby olan biri ile ilgili bir Karadeniz esprisi yapar mısın?': 'Chaby alabilirsin'
structured-chatbot-conversation:
{
    "messages": [
        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},
        {"role": "user", "content": "Adı Chaby olan biri ile ilgili bir Karadeniz esprisi yapar mısın?"},
        {"role": "assistant", "content": "Bir Karadenizliye masadaki sandalyeyi alabilir miyim diye sorduğunda ne der?"},
        {"role": "user", "content": "Ne der?"},
        {"role": "assistant", "content": "Chaby alabilirsin"}
    ]
}
q-a pair:
'Bir halı saha maçında antrenör Beyazıt Öztürk’e ‘bu gölü kim atacak?’ diye sorduğunda ne cevap vermiştir?': 'Hocam Ali Atay'
structured-chatbot-conversation:
{
    "messages": [
        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},
        {"role": "user", "content": " r harflerini söyleyemeyen Beyazıt Öztürk ile ilgili bir espri yapar mısın?"},
        {"role": "assistant", "content": "Bir halı saha maçında antrenör Beyazıt Öztürk’e ‘bu gölü kim atacak?’ diye sorduğunda ne cevap vermiştir?"},
        {"role": "user", "content": "Ne cevap vermiştir?"},
        {"role": "assistant", "content": "Hocam Ali Atay"}
    ]
}
"""

In [5]:
def dad_joke_sample_formatter(qa_pair,system_prompt):
    user_content = f"""
    q-a pair: 
    {qa_pair}
    structured-chatbot-conversation:
    """
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": user_content
            }
        ]
    )

    return completion.choices[0].message.content

In [ ]:
from tqdm import tqdm

dad_joke_samples = []
# Show progress
print("Converting q-a pairs to chatbot conversation format...")

for qa_pair in tqdm(qa_list):
    dad_joke_samples.append(dad_joke_sample_formatter(qa_pair, system_prompt))

In [62]:
#train-test split
train_samples = dad_joke_samples[:int(len(dad_joke_samples)*0.8)]
test_samples = dad_joke_samples[int(len(dad_joke_samples)*0.8):]

In [ ]:
train_samples[0]

In [63]:
import json
import re

def write_cleaned_jsonl(dad_joke_samples, output_file):
    with open(output_file, "w") as f:
        for joke in dad_joke_samples:
            data = json.loads(joke)
            cleaned_text = json.dumps(data, separators=(',', ':'), ensure_ascii=False)
            f.write(cleaned_text + "\n")

In [64]:
output_file = "dad_joke_samples_train.jsonl"
write_cleaned_jsonl(train_samples, output_file)

output_file = "dad_joke_samples_test.jsonl"
write_cleaned_jsonl(test_samples, output_file)

In [ ]:
#create jsonl file from the dad_joke_samples
#replace \n with ''

import json
import re
with open("dad_joke_samples.jsonl", "w") as f:
    for joke in dad_joke_samples:
        data = json.loads(joke)
        cleaned_text = json.dumps(data, separators=(',', ':'),ensure_ascii=False)
        #joke = joke.replace("\n","")
        #joke = re.sub(r'\s+', '', joke)
        f.write(cleaned_text + "\n")

In [44]:
dad_joke_samples[0]

'{\n    "messages": [\n        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},\n        {"role": "user", "content": "Bana bir espri yapar mısın?"}, \n        {"role": "assistant", "content": "Dilek tutmana yardımcı olacak meyve nedir?"}, \n        {"role": "user", "content": "Nedir?"}, \n        {"role": "assistant", "content": "WishNe."}\n    ]\n}'

In [46]:
dataset = [json.loads(line) for line in dad_joke_samples]
print("First example:")
for message in dataset[0]['messages']:
    print(message)

First example:
{'role': 'system', 'content': 'You are a dad joke expert. Create dad jokes in accordance with the user input.'}
{'role': 'user', 'content': 'Bana bir espri yapar mısın?'}
{'role': 'assistant', 'content': 'Dilek tutmana yardımcı olacak meyve nedir?'}
{'role': 'user', 'content': 'Nedir?'}
{'role': 'assistant', 'content': 'WishNe.'}


In [ ]:
{
    "messages": [
        {"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."},
        {"role": "user", "content": "Adı Chaby olan biri ile ilgili bir Karadeniz esprisi yapar mısın?"},
        {"role": "assistant", "content": "Reasoning: asdfasşnlşmbvdfVfd Espri sorusu: Bir Karadenizliye masadaki sandalyeyi alabilir miyim diye sorduğunda ne der?"},
        {"role": "user", "content": "Ne der?"},
        {"role": "assistant", "content": "Chaby alabilirsin"}
    ]
}